### Basic working of Google Palm LLM in LangChain

In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import sys
sys.path

['/home/gong/projects/AI/langchain/3_project_codebasics_q_and_a',
 '/home/gong/anaconda3/lib/python39.zip',
 '/home/gong/anaconda3/lib/python3.9',
 '/home/gong/anaconda3/lib/python3.9/lib-dynload',
 '',
 '/home/gong/projects/st/lib/python3.9/site-packages']

in jupyter

```
['/home/gong/projects/AI/langchain/3_project_codebasics_q_and_a',
 '/home/gong/anaconda3/lib/python39.zip',
 '/home/gong/anaconda3/lib/python3.9',
 '/home/gong/anaconda3/lib/python3.9/lib-dynload',
 '',
 '/home/gong/anaconda3/lib/python3.9/site-packages',
 '/home/gong/anaconda3/lib/python3.9/site-packages/IPython/extensions',
 '/home/gong/.ipython']
```

in ipython

```
In [4]: sys.path
Out[4]:
['/home/gong/projects/st/lib/python3.9/site-packages',
 '/home/gong/anaconda3/bin',
 '/home/gong/anaconda3/lib/python39.zip',
 '/home/gong/anaconda3/lib/python3.9',
 '/home/gong/anaconda3/lib/python3.9/lib-dynload',
 '',
 '/home/gong/anaconda3/lib/python3.9/site-packages',
 '/home/gong/anaconda3/lib/python3.9/site-packages/IPython/extensions',
 '/home/gong/.ipython']
```

In [3]:
from langchain.llms import GooglePalm

api_key = os.environ["GOOGLE_API_KEY"]
# 'your api key here' # get this free api key from https://makersuite.google.com/

llm = GooglePalm(google_api_key=api_key, temperature=0.1)

In [5]:
poem = llm("Write a 4 line poem of my love for samosa")
print(poem)

**Oh, samosa, my love**
**You are so crispy and golden brown**
**With your filling of spiced potatoes**
**You are the perfect snack**


In [6]:
essay = llm("write email requesting refund for electronic item")
print(essay)

Dear [Name of Customer Service Representative],

I am writing to request a refund for the [product name] that I purchased on [date]. I am not satisfied with the product because [explain why you are not satisfied].

I have attached a copy of my receipt and a picture of the product. I have also included a link to the product's listing on your website.

I would like to request a full refund for the product. I would also like to request that you send me a replacement product.

I have been a loyal customer of your company for many years, and I have never had a problem with a product before. I am disappointed that I am not satisfied with this product, but I am confident that you will be able to resolve this issue to my satisfaction.

Thank you for your time and attention to this matter.

Sincerely,
[Your Name]


In [7]:
from langchain.chains import RetrievalQA


from langchain.embeddings import GooglePalmEmbeddings
from langchain.llms import GooglePalm

### Now let's load data from Codebasics FAQ csv file

In [8]:
!ls codebasics_faqs.csv

codebasics_faqs.csv


In [10]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='codebasics_faqs.csv', source_column="prompt")

In [11]:
# Store the loaded data in the 'data' variable
data = loader.load()

### Hugging Face Embeddings

In [12]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

# Initialize instructor embeddings using the Hugging Face model
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large")

e = instructor_embeddings.embed_query("What is your refund policy?")

load INSTRUCTOR_Transformer
max_seq_length  512


In [13]:
len(e)

768

In [14]:
e[:5]

[-0.0438980795443058,
 0.007685551419854164,
 -0.009231897071003914,
 0.024496253579854965,
 0.03359227254986763]

As you can see above, embedding for a sentance "What is your refund policy" is a list of size 768. Looking at the numbers in this list, doesn't give any intuitive understanding of what it is but just assume that these numbers are capturing the meaning of "What is your refund policy". If you are curious to know about embeddings, go to youtube and search "codebasics word embeddings" and you will find bunch of videos with simple, intuitive explanations

### Vector store using FAISS

In [16]:
from langchain.vectorstores import FAISS

# Create a FAISS instance for vector database from 'data'
vectordb = FAISS.from_documents(documents=data,
                                 embedding=instructor_embeddings)

# Create a retriever for querying the vector database
retriever = vectordb.as_retriever(score_threshold = 0.7)

In [17]:
rdocs = retriever.get_relevant_documents("how about job placement support?")
rdocs

[Document(page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.', metadata={'source': 'Do you provide any job assistance?', 'row': 11}),
 Document(page_content='prompt: Will this course guarantee me a job?\nresponse: We created a much lighter version of this course on YouTube available for free (click this link) and many people gave us feedback that they were able to fetch jobs (see testimonials). Now this paid course is at least 5x better than the YouTube course which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.', metadata={'source': 'Will this 

In [24]:
len(rdocs), rdocs[0], type(rdocs[0]), rdocs[0].dict().keys()

(4,
 Document(page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.', metadata={'source': 'Do you provide any job assistance?', 'row': 11}),
 langchain.schema.document.Document,
 dict_keys(['page_content', 'metadata', 'type']))

In [25]:
rdocs[0].page_content

'prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.'

As you can see above, the retriever that was created using FAISS and hugging face embedding is now capable of pulling relavant documents from our original CSV file knowledge store. This is very powerful and it will help us further in our project

##### Embeddings can be created using GooglePalm too. Also for vector database you can use chromadb as well as shown below. During our experimentation, we found hugging face embeddings and FAISS to be more appropriate for our use case

In [14]:
# google_palm_embeddings = GooglePalmEmbeddings(google_api_key=api_key)

# from langchain.vectorstores import Chroma
# vectordb = Chroma.from_documents(data,
#                            embedding=google_palm_embeddings,
#                            persist_directory='./chromadb')
# vectordb.persist()

### Create RetrievalQA chain along with prompt template 🚀

In [26]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}


from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs)


### We are all set 👍🏼 Let's ask some questions now

In [27]:
resp = chain('Do you provide job assistance and also do you provide job gurantee?')
resp

{'query': 'Do you provide job assistance and also do you provide job gurantee?',
 'result': 'Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters. The courses included in this bootcamp are done by 9000+ learners and many of them have secured a job which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.',
 'source_documents': [Document(page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.', metadata={'source': 'Do you prov

**As you can see above, the answer of question comes from two different FAQs within our csv file and it is able to pull those questions and merge them nicely**

In [28]:
resp = chain("Do you guys provide internship and also do you offer EMI payments?")
resp

{'query': 'Do you guys provide internship and also do you offer EMI payments?',
 'result': "Yes, we provide virtual internship. No, we don't offer EMI payments.",
 'source_documents': [Document(page_content='prompt: Do you provide any virtual internship?\nresponse: Yes', metadata={'source': 'Do you provide any virtual internship?', 'row': 14}),
  Document(page_content='prompt: Do we have an EMI option?\nresponse: No', metadata={'source': 'Do we have an EMI option?', 'row': 13}),
  Document(page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.', metadata={'source': 'Do you provide any job assistance?', 'row': 11}),
  Document(page_content='prompt: How can I contact the instructors for any doubts/support?\nresponse: We have created every lecture with a motive to explain everything in an easy

In [29]:
chain("do you have javascript course?")

{'query': 'do you have javascript course?',
 'result': "I don't know.",
 'source_documents': [Document(page_content='prompt: I have never done programming and belong to a non-technical background. Can I take this course?\nresponse: Yes, this is the perfect course for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in their current job or business using data.', metadata={'source': 'I have never done programming and belong to a non-technical background. Can I take this course?', 'row': 24}),
  Document(page_content='prompt: I have never done programming in my life. Can I take this bootcamp?\nresponse: Yes, this is the perfect bootcamp for anyone who has never done coding and wants to build a career in the IT/Data Analytics industry or just wants to perform better in your current job or business using data.', metadata={'source': 'I have never done programming in my life. Can I take this bootcamp?', 'row': 0}),


In [30]:
chain("Do you have plans to launch blockchain course in future?")

{'query': 'Do you have plans to launch blockchain course in future?',
 'result': "I don't know.",
 'source_documents': [Document(page_content='prompt: Will the course be upgraded when there are new features in Power BI?\nresponse: Yes, the course will be upgraded periodically based on the new features in Power BI, and learners who have already bought this course will have free access to the upgrades.', metadata={'source': 'Will the course be upgraded when there are new features in Power BI?', 'row': 27}),
  Document(page_content='prompt: What business concepts and domains are covered in this course?\nresponse: We have covered the core functions such as Sales, Marketing, Finance, and Supply Chain with their fundamentals related to this course. The domain you will learn in this course is consumer goods which is projected to have more openings and high data analytics requirements at least until 2030.', metadata={'source': 'What business concepts and domains are covered in this course?', '

In [31]:
chain("should I learn power bi or tableau?")

{'query': 'should I learn power bi or tableau?',
 'result': "This is a contextual question. If you are talking about a pure visualization tool Tableau is slightly better. Data connectors, modeling and transformation features are available in both. However, factually speaking Power BI is cheaper and offers tighter integration with the Microsoft environment. Since most companies use excel & Microsoft tools they start with Power BI or move towards Power BI for seamless integration with other Microsoft tools (called as Power platform). This makes the job openings grow at a much higher rate on Power BI and Power Platform. Also, Power BI has been leading the Gartner's magic quadrant in BI for the last few years as the industry leader.",
 'source_documents': [Document(page_content="prompt: Power BI or Tableau which one is better?\nresponse: This is a contextual question. If you are talking about a pure visualization tool Tableau is slightly better. Data connectors, modeling and transformation

In [32]:
chain("I've a MAC computer. Can I use powerbi on it?")

{'query': "I've a MAC computer. Can I use powerbi on it?",
 'result': 'response: Power BI desktop works only in Windows OS. Please look into the system requirements section on this page. However, you can use a virtual machine to install and work with Power BI in other Operating systems.',
 'source_documents': [Document(page_content='prompt: How can I use PowerBI on my Mac system?\nresponse: Hi\r\n\r\nYou can use VirtualBox to create a virtual machine and install Windows on it. This will allow you to run Power BI and Excel on your Mac.\r\n\r\nIf you\'re not familiar with setting up a virtual machine, there are many resources available on YouTube that can guide you through the process. Simply search for "installing virtual machines" and you\'ll find plenty of helpful videos.\r\n\r\nBest of luck with your studies!', metadata={'source': 'How can I use PowerBI on my Mac system?', 'row': 44}),
  Document(page_content='prompt: Does Power BI work in Mac OS/Ubuntu?\nresponse: Power BI desktop w

In [33]:
chain("I don't see power pivot. how can I enable it?")

{'query': "I don't see power pivot. how can I enable it?",
 'result': 'Follow this thread for instructions - https://support.microsoft.com/en-us/office/start-the-power-pivot-add-in-for-excel-a891a66d-36e3-43fc-81e8-fc4798f39ea8',
 'source_documents': [Document(page_content='prompt: How to install power pivot if its not available in system?\nresponse: Follow this thread for instructions - https://support.microsoft.com/en-us/office/start-the-power-pivot-add-in-for-excel-a891a66d-36e3-43fc-81e8-fc4798f39ea8\r\nIf it doesn\'t show in the ribbon then go to "insert" tab. You will be able to see pivot table option there.', metadata={'source': 'How to install power pivot if its not available in system?', 'row': 38}),
  Document(page_content='prompt: How do I enable Power Pivot before using it for the first time ?\nresponse: Follow the process in the link : \r\n\r\nhttps://drive.google.com/file/d/1-mO-v52h-YTY1s-v30liBJPu6Yj4OUxb/view?usp=share_link', metadata={'source': 'How do I enable Power 

In [34]:
chain("What is the price of your machine learning course?")

{'query': 'What is the price of your machine learning course?',
 'result': "I don't know.",
 'source_documents': [Document(page_content="prompt: I'm not sure if this course is good enough for me to invest some money. What can I do?\nresponse: Don't worry. Many videos in this course are free so watch them to get an idea of the quality of teaching. Dhaval Patel (the course instructor) runs a popular data science YouTube channel called Codebasics. On that, you can watch his videos and read comments to get an idea of his teaching style", metadata={'source': "I'm not sure if this course is good enough for me to invest some money. What can I do?", 'row': 20}),
  Document(page_content='prompt: Will the course be upgraded when there are new features in Power BI?\nresponse: Yes, the course will be upgraded periodically based on the new features in Power BI, and learners who have already bought this course will have free access to the upgrades.', metadata={'source': 'Will the course be upgraded 

In [35]:
chain("What is Khan Academy?")

{'query': 'What is Khan Academy?',
 'result': "I don't know.",
 'source_documents': [Document(page_content="prompt: I'm not sure if this course is good enough for me to invest some money. What can I do?\nresponse: Don't worry. Many videos in this course are free so watch them to get an idea of the quality of teaching. Dhaval Patel (the course instructor) runs a popular data science YouTube channel called Codebasics. On that, you can watch his videos and read comments to get an idea of his teaching style", metadata={'source': "I'm not sure if this course is good enough for me to invest some money. What can I do?", 'row': 20}),
  Document(page_content="prompt: What is different in this course compared to hundreds of courses on the internet and free tutorials on YouTube?\nresponse: Most of the courses available on the internet teach you how to build x & y without any business context and do not prepare you for real business world problem-solving. This course is rather an experience in whi